# Visualización de Incendios Forestales

In [228]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [229]:
# Dataframe
url = 'https://raw.githubusercontent.com/AndrsGzRo/incendios-forestales-mx/refs/heads/main/data-2025-11-05.csv'
df = pd.read_csv(url)

# Primeros cincos renglones
df.head()

,anio,Clave_del_incendio,latitud,longitud,Clave_Municipio,Estado,Municipio,CVE_ENT,CVE_MUN,CVEGEO,...,Entidad,fn_Clave_del_incendio,fn_Predio,fn_Causa,fn_Causa_especifica,fn_Duracion_dias,fn_Tipo_de_incendio,fn_Tipo_Vegetacion,fn_Tipo_impacto,fn_Tamano
0,2015,15-01-0001,21.995503,-102.750478,3.0,Aguascalientes,Calvillo,1,3.0,301,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,15-01-0002,21.773878,-102.300881,1.0,Aguascalientes,Aguascalientes,1,1.0,101,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,15-01-0003,21.785075,-102.319444,1.0,Aguascalientes,Aguascalientes,1,1.0,101,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,15-01-0004,22.224067,-102.336411,7.0,Aguascalientes,Rincón de Romos,1,7.0,701,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,15-01-0005,22.041083,-101.973917,2.0,Aguascalientes,Asientos,1,2.0,201,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.1 Información de dataframe

In [230]:
def info_df(df):
    print('---------Información general del Dataframe---------')
    print(df.info())
    print('---------Estadísticas del df---------')
    print(df.describe())
    print('---------Valores nulos---------')
    print(df.isnull().sum().sort_values(ascending=False))
    print('---------Valores únicos---------')
    print(df.nunique())

info_df(df)

---------Información general del Dataframe---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71089 entries, 0 to 71088
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   anio                   71089 non-null  int64  
 1   Clave_del_incendio     71089 non-null  object 
 2   latitud                71089 non-null  float64
 3   longitud               71089 non-null  float64
 4   Clave_Municipio        71075 non-null  float64
 5   Estado                 71089 non-null  object 
 6   Municipio              71089 non-null  object 
 7   CVE_ENT                71089 non-null  int64  
 8   CVE_MUN                71089 non-null  float64
 9   CVEGEO                 71089 non-null  int64  
 10  Region                 71089 non-null  object 
 11  Predio                 71087 non-null  object 
 12  Causa                  71089 non-null  object 
 13  Causa_especifica       71089 non-null  object 
 14  Fe

In [231]:
df[['Fecha_Inicio','Fecha_Termino']]

,Fecha_Inicio,Fecha_Termino
0,16439.0,16440
1,16449.0,16449
2,16449.0,16450
3,16458.0,16458
4,16458.0,16459
...,...,...
71084,19855.0,19855
71085,19853.0,19864
71086,19873.0,19874
71087,20043.0,20043


## 1.2 Limpieza

Se eliminarán las variables que empiezan con ``` fn_``` ya que suelen ser valores nulos.




In [232]:
# Eliminando columnas fn
df = df.copy()
df_n= df.loc[:,~df.columns.str.startswith('fn_')].copy()
info_df(df_n)

---------Información general del Dataframe---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71089 entries, 0 to 71088
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   anio                71089 non-null  int64  
 1   Clave_del_incendio  71089 non-null  object 
 2   latitud             71089 non-null  float64
 3   longitud            71089 non-null  float64
 4   Clave_Municipio     71075 non-null  float64
 5   Estado              71089 non-null  object 
 6   Municipio           71089 non-null  object 
 7   CVE_ENT             71089 non-null  int64  
 8   CVE_MUN             71089 non-null  float64
 9   CVEGEO              71089 non-null  int64  
 10  Region              71089 non-null  object 
 11  Predio              71087 non-null  object 
 12  Causa               71089 non-null  object 
 13  Causa_especifica    71089 non-null  object 
 14  Fecha_Inicio        71086 non-null  float64
 15  F

## 1.3 Transformación de Variables

### 1.3.1 Duracion_dias
Las variables anteriores tienen un formato correcto, excepto ```Duracion_dias```, la cual será transformada a una ordinal.


In [233]:
mapa_duraciones = {
    '1 Día': 1,
    '2 a 3 Días': 2.5,
    '4 a 7 Días':5.5,
    'Mayor de 7 Días':10
}
df['Duracion_dias_num'] = df['Duracion_dias'].map(mapa_duraciones)

### 1.3.2 Fecha_Inicio y Fecha_Termino
Al estar en TimeStamp es necesario transformar las variables a DateTime para comprender mejor la duración de los incendios.

In [234]:
# Fecha Inicio
df['Fecha_Inicio'] = pd.to_datetime('1970-01-01') + pd.to_timedelta(df['Fecha_Inicio'], unit='D')

# Fecha Termino
df['Fecha_Termino'] = pd.to_datetime('1970-01-01') + pd.to_timedelta(df['Fecha_Termino'], unit='D')

# Año, mes y duracion
df['anio_inicio'] = df['Fecha_Inicio'].dt.year.astype(int)
df['mes_inicio'] = df['Fecha_Inicio'].dt.month
df['duracion_real'] = (df['Fecha_Termino'] - df['Fecha_Inicio']).dt.days

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df['año_inicio']